In [5]:
import pandas as pd
from sodapy import Socrata
import os

 
TOKEN_APP = "xv66xLFwvXNRjMns6TbebGarR"
 
client = Socrata("www.datos.gov.co", TOKEN_APP)
 
 

In [6]:
socrata_data_identifier = "jbjy-vk9h"
 
Query = """
select
    id_contrato, nombre_entidad, departamento, modalidad_de_contratacion, valor_del_contrato, fecha_de_firma,proveedor_adjudicado
where
    nombre_entidad LIKE 'UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP'
limit
10000000
"""
 
 
result = client.get(socrata_data_identifier, query = Query)
 
result_df = pd.DataFrame.from_records(result)

In [7]:
print(result_df.dtypes)

id_contrato                  object
nombre_entidad               object
departamento                 object
modalidad_de_contratacion    object
valor_del_contrato           object
fecha_de_firma               object
dtype: object


In [8]:
result_df['valor_del_contrato'] = result_df ['valor_del_contrato'].astype(float)
result_df['fecha_de_firma'] = pd.to_datetime(result_df['fecha_de_firma'], format='mixed')

In [9]:
print(result_df.dtypes)

id_contrato                          object
nombre_entidad                       object
departamento                         object
modalidad_de_contratacion            object
valor_del_contrato                  float64
fecha_de_firma               datetime64[ns]
dtype: object


In [10]:
result_df

,id_contrato,nombre_entidad,departamento,modalidad_de_contratacion,valor_del_contrato,fecha_de_firma
0,CO1.PCCNTR.2193149,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,41523626.0,2021-02-01
1,CO1.PCCNTR.5982280,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,8927010.0,2024-02-21
2,CO1.PCCNTR.280189,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,8400000.0,2018-01-11
3,CO1.PCCNTR.4376829,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,23087052.0,2023-01-06
4,CO1.PCCNTR.277084,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,16480000.0,2018-01-09
...,...,...,...,...,...,...
2517,CO1.PCCNTR.2115837,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,335358912.0,2021-01-14
2518,CO1.PCCNTR.271971,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,9600000.0,2018-01-05
2519,CO1.PCCNTR.2168350,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,41050116.0,2021-01-25
2520,CO1.PCCNTR.4622885,UNIDAD DE GESTION PENSIONAL Y PARAFISCALES - UGPP,Distrito Capital de Bogotá,Contratación directa,31800000.0,2023-02-14


In [11]:
print(result_df.columns)

Index(['id_contrato', 'nombre_entidad', 'departamento',
       'modalidad_de_contratacion', 'valor_del_contrato', 'fecha_de_firma'],
      dtype='object')
